### Importing all necessary libraries for web scraping

In [132]:
import pandas as pd
import numpy as np
import requests
import json
import time

### Setting URLs for the subreddits

In [133]:
# creating URLs and header so that the reddit api responds
url1 = 'https://www.reddit.com/r/leagueoflegends/.json'
url2 = 'https://www.reddit.com/r/DotA2.json'
headers = {'User-agent': 'stan-sham'}

In [134]:
# creating access to reddit API
res1 = requests.get(url1, headers = headers)
res2 = requests.get(url2, headers = headers)

In [135]:
# Checking to see that this is a valid subreddit
print(res1.status_code)
print(res2.status_code)

200
200


In [136]:
# Following function code was provided by Boom Devahastin Na Ayudhya
# Function to pull subreddit posts
def get_subreddit(url, n_pulls, headers):    

    # Create empty templates
    posts = []
    after = None

    # Create a loop that does max 25 requests per pull
    for pull_num in range(n_pulls):
        print("Pulling data attempted", pull_num+1,"time(s)")

        if after == None:
            new_url = url                 # base case
        else:
            new_url = url+"?after="+after # subsequent iterations

        res = requests.get(new_url, headers=headers)

        if res.status_code == 200:
            subreddit_json = res.json()                      # Pull JSON
            posts.extend(subreddit_json['data']['children']) # Get subreddit posts
            after = subreddit_json['data']['after']          # 'after' = ID of the last post in this iteration
        else:
            print("We've run into an error. The status code is:", res.status_code)
            break

        time.sleep(1)
        
    return(posts)

# Doesn't give dataframe, gotta make your own

In [137]:
# Pulling league of legends data into list for easier manipulation
leagueList = get_subreddit(url1, 60, headers)

Pulling data attempted 1 time(s)
Pulling data attempted 2 time(s)
Pulling data attempted 3 time(s)
Pulling data attempted 4 time(s)
Pulling data attempted 5 time(s)
Pulling data attempted 6 time(s)
Pulling data attempted 7 time(s)
Pulling data attempted 8 time(s)
Pulling data attempted 9 time(s)
Pulling data attempted 10 time(s)
Pulling data attempted 11 time(s)
Pulling data attempted 12 time(s)
Pulling data attempted 13 time(s)
Pulling data attempted 14 time(s)
Pulling data attempted 15 time(s)
Pulling data attempted 16 time(s)
Pulling data attempted 17 time(s)
Pulling data attempted 18 time(s)
Pulling data attempted 19 time(s)
Pulling data attempted 20 time(s)
Pulling data attempted 21 time(s)
Pulling data attempted 22 time(s)
Pulling data attempted 23 time(s)
Pulling data attempted 24 time(s)
Pulling data attempted 25 time(s)
Pulling data attempted 26 time(s)
Pulling data attempted 27 time(s)
Pulling data attempted 28 time(s)
Pulling data attempted 29 time(s)
Pulling data attempted 

In [138]:
# pulling apex legends data into list
dotaList = get_subreddit(url2, 80, headers)

Pulling data attempted 1 time(s)
Pulling data attempted 2 time(s)
Pulling data attempted 3 time(s)
Pulling data attempted 4 time(s)
Pulling data attempted 5 time(s)
Pulling data attempted 6 time(s)
Pulling data attempted 7 time(s)
Pulling data attempted 8 time(s)
Pulling data attempted 9 time(s)
Pulling data attempted 10 time(s)
Pulling data attempted 11 time(s)
Pulling data attempted 12 time(s)
Pulling data attempted 13 time(s)
Pulling data attempted 14 time(s)
Pulling data attempted 15 time(s)
Pulling data attempted 16 time(s)
Pulling data attempted 17 time(s)
Pulling data attempted 18 time(s)
Pulling data attempted 19 time(s)
Pulling data attempted 20 time(s)
Pulling data attempted 21 time(s)
Pulling data attempted 22 time(s)
Pulling data attempted 23 time(s)
Pulling data attempted 24 time(s)
Pulling data attempted 25 time(s)
Pulling data attempted 26 time(s)
Pulling data attempted 27 time(s)
Pulling data attempted 28 time(s)
Pulling data attempted 29 time(s)
Pulling data attempted 

In [139]:
# Creating function to separate title, text, and the subreddit that its from
def convertSubreddit(subreddit):
    titleList = []
    textList = []
    subredditList = []
    for i in range(len(subreddit)):
        titleList.append(subreddit[i]['data']['title'])
        textList.append(subreddit[i]['data']['selftext'])
        subredditList.append(subreddit[i]['data']['subreddit'])
        
    newDF = pd.DataFrame(columns = ['Titles', 'Text', 'Subreddit'])
    newDF['Titles'] = titleList
    newDF['Text'] = textList
    newDF['Subreddit'] = subredditList

    return (newDF)

In [149]:
# converting lists to dataframs
leagueDF = convertSubreddit(leagueList)
dotaDF = convertSubreddit(dotaList)

In [153]:
# checking that league data has been converted correctly
leagueDF.head()

,Titles,Text,Subreddit
0,Ready to prove you're Challenger? /r/LeagueofL...,"Hi Summoners,\n\nDrum roll please... we have s...",leagueoflegends
1,Patch 9.7 Bug Megathread,Greetings Summoners!\n\nWith every new patch R...,leagueoflegends
2,Mediocre Rap Battle of #LEC｜G2 vs Origen,,leagueoflegends
3,Add the ability to remove all client animations.,Having to wait a duration longer than a Morgan...,leagueoflegends
4,You know a champion is old when they still tal...,"Udyr, Cho’gath and Mordekaiser are all pretty ...",leagueoflegends


In [154]:
# checking that apex data has been converted correctly
dotaDF.head()

,Titles,Text,Subreddit
0,OGA Dota PIT Minor 2019: CIS Qualifier - Day 2...,# [](/org-dotapit) [OGA Dota PIT Minor 2019 - ...,DotA2
1,OGA Dota PIT Minor 2019: North &amp; South Ame...,# [](/org-dotapit) [OGA Dota PIT Minor 2019 - ...,DotA2
2,Chase your dreams,,DotA2
3,We never forget our first time,,DotA2
4,Which combination do you like the best?,,DotA2


In [155]:
# There are posts which are just videos or images, we will be dropping those entries 
# since they do not have any relevance to this project
mask = dotaDF['Text'] == ''
dotaDF.drop(dotaDF[mask].index, inplace = True)

In [156]:
# Checking size of new dataframe
dotaDF.shape

(1035, 3)

In [157]:
# dropping entries with no text for league of legends subreddit
mask = leagueDF['Text'] == ''
leagueDF.drop(leagueDF[mask].index, inplace = True)

In [158]:
# Checking size of new dataframe
leagueDF.shape

(1118, 3)

Since both dataframes are relatively the same size, we can use both of these dataframes for EDA

Titles       0
Text         0
Subreddit    0
dtype: int64

In [159]:
# creating one large dataframe with both subreddit data
redditDF = pd.concat([leagueDF, dotaDF])

In [161]:
# Creating CSV files for each data frame
redditDF.to_csv('./reddit_dataframe.csv')
leagueDF.to_csv('./league_dataframe.csv')
dotaDF.to_csv('./dota_dataframe.csv')